In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import distance_matrix
from time import monotonic
from datetime import datetime
#print(repr(d.astype(int)))

def plot_co(line):
    # Parse the input text and extract data
    input_text = line

    lines = input_text.strip().split('output')
    coordinates = [list(map(float, line.split())) for line in lines]

    # Extract x and y coordinates
    x_coords = coordinates[0][::2]
    y_coords = coordinates[0][1::2]
    data={'x': x_coords, 'y': y_coords}
    df=pd.DataFrame(data)
    n_point=len(df)
    time_step=n_point

    d = distance_matrix(df[['x', 'y']].values, df[['x', 'y']].values, p=2)  # 1=manhattan distance, 2=euclidean distance
    d = (repr(d))
    d = d.strip("array()\n")
    return d

In [3]:
import re

# Read the text from a file (replace 'your_file.txt' with the actual file name)
file_path='/mnt/f/year4/senior/kmutt_tsp_50_data.txt'
#file_path='kmutt_tsp_10_data.txt'
N_samples=-1

# Function to process a line and calculate the sum
def process_line(line):
    numbers = [float(match.group()) for match in re.finditer(r'\b\d+(\.\d+)?\b', line)]
    result = plot_co(line)
    with open(file_path, 'r') as file:
        # Read each line and process it
        count = 0
        for line in file:
            count = count + 1
            result_line = process_line(line)
            data = create_data_model(result_line)
            manager = pywrapcp.RoutingIndexManager(
            len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
            )
            routing = pywrapcp.RoutingModel(manager)
            transit_callback_index = routing.RegisterTransitCallback(distance_callback)
            routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
            search_parameters = pywrapcp.DefaultRoutingSearchParameters()
            search_parameters.first_solution_strategy = (
            routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING
            )
            search_parameters.time_limit.seconds = 300
            search_parameters.log_search = True
            solution = routing.SolveWithParameters(search_parameters)
            if solution:
                print_solution(manager, routing, solution)
            if count > 2 : break
            print(data)
    return f"{result}"

# 28 JAN 2024 func
def read_tsp_instances_from_file(filename):
    instances=[]
    with open(filename, 'r') as file:
        # read the first line to get N-Sample and n_point
        first_line=file.readline().strip()
        _, _, num_samples_str, _, _, num_points_str = first_line.split()
        global N_samples
        N_samples, num_samples=int(num_samples_str), int(num_samples_str)
        num_points=int(num_points_str)

        # loop through each N-Sample
        for i in range(num_samples):
            line=file.readline().strip()
            data=list(map(float, line.split()[:-1]))
            data_denormalize=[round(i*1e16) for i in data]  # OR-Tools does not take into account floating point values for the distance matrix!!
            data=data_denormalize

            coordinates=np.array(data).reshape((num_points, 2)) # extract x and y coordinates
            distance_matrix_np=distance_matrix(coordinates, coordinates)
            instances.append(distance_matrix_np)
    return instances

tsp_instances=read_tsp_instances_from_file(file_path)

In [4]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model(matrix):
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"]=matrix
    data["num_vehicles"] = 1
    data["depot"] = 0
#    print("infunction\n" + data["distance_matrix"])
    return data


def print_solution(manager, routing, solution):
    """Prints solution on console."""
    path_list = []
    #print(f"Objective: {solution.ObjectiveValue()} miles")
    index = routing.Start(0)
    plan_output = "Route for vehicle 0:\n"
    route_distance = 0
    while not routing.IsEnd(index):
        path_list.append(index)
        plan_output += f" {manager.IndexToNode(index)} ->"
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    path_list.append(index%len(path_list))
    plan_output += f" {manager.IndexToNode(index)}\n"
    #print(plan_output)
    plan_output += f"Route distance: {route_distance}miles\n"
    return path_list

def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes

In [5]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data["distance_matrix"][from_node][to_node]

#transit_callback_index = routing.RegisterTransitCallback(distance_callback)

In [6]:
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    all_path_list=[]
    start_time = monotonic()
    start = datetime.now()
    for i in range(N_samples):
        data = create_data_model(tsp_instances[i].astype(int))

        # Create the routing index manager.
        manager = pywrapcp.RoutingIndexManager(
            len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
        )

        # Create Routing Model.
        routing = pywrapcp.RoutingModel(manager)


        def distance_callback(from_index, to_index):
            """Returns the distance between the two nodes."""
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data["distance_matrix"][from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
        )
        start_time = monotonic()
        # Solve the problem.
        solution = routing.SolveWithParameters(search_parameters)
        
        # Print solution on console.
        if solution:
            #print("Line", i, "route:")
            path_list=print_solution(manager, routing, solution)
            
        #path_list=get_routes(manager, routing, solution)
        all_path_list.append(path_list)
        #all_path_list = get_routes(manager, routing, solution)
    end = datetime.now()
    td = (end - start).total_seconds() #* 10**3
    print(f"The time of execution of above program is : {td:.03f}s")
    print(f"Solution time {monotonic() - start_time} seconds")    
    return all_path_list

if __name__=="__main__":
    routes=main()

The time of execution of above program is : 4093.935s
Solution time 0.03171340000699274 seconds


In [ ]:
print(routes[0])

In [7]:
' '.join(map(str,routes[0]))

'0 47 49 29 32 11 3 25 37 5 35 44 45 42 10 17 19 27 13 7 8 38 23 21 31 1 36 12 33 18 20 39 2 22 48 24 16 43 34 46 41 6 28 4 9 40 30 15 26 14 0'

In [8]:
file_path_2 = '/mnt/f/year4/senior/kmutt_tsp_50_data - Copy.txt'
# Open the input file in read mode
with open(file_path_2, 'r') as file:
    # Read all lines from the file
    lines = file.readlines()

# Open the same file in write mode
with open(file_path_2, 'w') as file:
    # Iterate through each line
    firstline = False
    for i, line in enumerate(lines):
        # Skip the first line
        if not firstline: firstline = True
        # Strip newline character from the line and append your list with numbers separated by space
        else: line = line.strip() + " " + " ".join(map(str, routes[i-1])) + "\n"
        # Write the modified line back to the file
        file.write(line)